In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows',5000)
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
df2 =pd.read_csv("students_final2.csv").copy()
df2.loc[len(df2.index)] = ['anonim_mis',"MIS463","Aslı Sencer","anonim_mis MIS463 dersini Aslı Sencer hocasından kesinlikle öneriyor.",1.0]

In [3]:
df2.loc[df2['Text'].str.contains('hocasından şiddetle önermiyor.'), 'Text'] = "-"
df2.loc[df2['Text'].str.contains('hocasından kesinlikle öneriyor.'), 'Text'] = "-"
df2.loc[df2['Text'].str.contains('hocasından ne iyi ne kötü buluyor.'), 'Text'] = "-"

In [4]:
df2.head()

,Username,Lesson Name,Teacher Name,Text,Point
0,elif,SOC101,Ali Tuna Kuyucu,-,0.0
1,Zeynep837,SOC101,Ali Tuna Kuyucu,-,0.0
2,mrymsygili,SOC101,Ali Tuna Kuyucu,-,0.0
3,cengin,SOC101,Ali Tuna Kuyucu,-,0.0
4,tylerdurden,SOC101,Ali Tuna Kuyucu,-,0.0


In [5]:
df=df2[df2.Text != "-"].copy()

In [6]:
import transformers as trf

bert = trf.TFBertModel.from_pretrained('dbmdz/bert-base-turkish-128k-uncased')

Some layers from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [7]:
tokenizer = trf.BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-128k-uncased', do_lower_case=True)

In [8]:
txt = "inanılmaz verimsiz bir deneme yorumu test edilecek"
## tokenize
idx = tokenizer.encode(txt)
print("tokens:", tokenizer.convert_ids_to_tokens(idx))
print("ids   :", tokenizer.encode(txt))

tokens: ['[CLS]', 'inanılmaz', 'verimsiz', 'bir', 'deneme', 'yorumu', 'test', 'edilecek', '[SEP]']
ids   : [2, 8890, 41824, 1947, 9222, 12742, 4429, 6728, 3]


In [9]:
idx = np.array(idx)[None,:] 
embedding = bert(idx)  
print("shape:", embedding[0].shape) 
                            

shape: (1, 9, 768)


In [10]:
positive_competence = ["Anlattığı konuya oldukça hakim.","Oldukça ufuk açıcı bir ders","Bu ders konuya dair oldukça öğreticiydi.","Kesinlikle bu dersi alabileceğiniz en iyi hocalardan birisi.","Her yönden çok donanımlı bir eğitmen. Öğrenci dostu olmasının yanında bilgili, bilgisini aktarmaya hevesli ve öğretmenlik motivasyonunu kaybetmemiş hocalardan.","Okulun en tatlı ve en kültürlü hocasıdır.", "gayet ilgili ve anlatım konusunda da oldukça başarılı","Dersleri hala büyük bir hevesle anlatıp ilginizi kaybetmemenizi sağlıyor.","dersleri dinlerken hiç sıkıldığımı hatırlamıyorum","Dersler esprili, komik ve eğlenceli geçiyor."]
negative_competence = ["Ders anlatımı hiç yetkin değil","Anlattığı dersi takip etmekte çok zorlanıyorum ve bu ders takip edilemiyor","Anlattıklarını kendi bile bilmiyor","bu dersi bilhassa bu hocadan almamanin bi formulunu bulmanizi oneririm.","kendisinden ders alınacak adam değildir.","hayatımda gördüğm en sorumsuz insanlardan biri.","Bir insan verdiği dersi nasıl bu kadar umursamaz aklım almıyor.","ders videolarını ve slaytları ya çok geç yükler ya eksik yükler.","Dersler asla takip edilemiyor","discussionlar sıkıcı","allah bu dersi alanlara sabır versin","dünyanın en korkunç dersi","tam anlamıyla hayatımı karartan bir ders oldu"]
student_friendly = ["Dersleri eğlenceli geçer.","İnanılmaz tatlı bir hoca","Kendisiyle istediğimiz zamanda iletişimde bulunabiliyoruz.","Maillere anında dönüyor.","Oldukça güler yüzlü ve neşeli birisi.","Ses tonu bile insanın içini ısıtıyor.","Kimsenin sorusunu cevapsız bırakmadı.","Tam anlamıyla öğrenci dostu."," Derslerinde ve hatta pslerde kayıt aldı ve dönem boyunca tüm ders materyallerini bizimle paylaştı.","kitabı gerekli bölümleri okuyorsunuz sonra yarın sınava giriyorsunuz ve AA geliyor","Ne zaman neye ihtiyacım olsa erinmeden dinledi yardım etti.","Her şeyini öğrenciler için feda edebilecek birisi tamamen mükemmellik abidesi"]
not_student_friendly = ["Bu ders sayesinde sigaraya başladım.","Neredeyse panik atak geçiriyordum.","Sınav sonuçlarını asla zamanında açıklamaz","Baya agrasif bir hoca","Öğrenciye her türkü zorluk çıkartabilecek birisi.","Her türlü mağduriyeti yaşarsınız.","Tam bir cadıdır kendisi.","diğer sectionların aksine katbekat zor sorar ve sınav ortalaması düşüktür.","Notu kıt bir hoca bence.","Seçmeme imkanı olan bir bölümdeyseniz yanından geçmeyin derim.","almayın yani korkunç bi ders"," Sınav değerlendirmeleri biraz saçma","notları kıt","sizi mağdur etmenin mutlaka bir yolunu bulur","öğrenciye tamamen düşman birisi","bu dersi kesinlikle önermiyorum"]
good_english = ["İnanılmaz akıcı bir ingilizceye sahip","Derslerine konuşmasını dinlemek için bile gidilir.","Derste söylediği cümleler gayet anlaşılabilir.","Tam bir beyefendi","Yurtdışında eğitim aldığı belli oluyor","anlattığı ders anlaşılabilir ve oldukça akıcı","türk aksanına yakın konuştuğu için dersleri anlaşılabilir","tane tane konuşuyor"]
bad_english = ["Ders esnasında söyledikleri hiç anlaşılmıyor.","Çok kötü bir aksanı var.","Derste söylenenlerden hiçbir şey anlayamıyorum.","Telafuzu çok kötü.","konuşmaya başladığı zaman insan direkt bunalıyor","ben bu kadar yorucu şekilde konuşan birisini hiç görmedim","bu adam nasıl native speaker anlamak güç","dersten hiçbir şey anlaşılmıyor"]

In [11]:
embedding[0]

<tf.Tensor: shape=(1, 9, 768), dtype=float32, numpy=
array([[[-0.52085036, -0.38367802, -2.135634  , ...,  0.13875636,
          0.25557533,  0.45156914],
        [ 0.15423784, -0.5098587 , -0.93862164, ..., -1.7609076 ,
         -0.07611521, -0.80006295],
        [-0.09277726, -0.00352648, -0.38873488, ..., -0.8360086 ,
          1.1227279 , -0.10558483],
        ...,
        [-0.87440735, -0.6651399 , -0.7555843 , ..., -0.44479614,
         -0.4471798 ,  0.38828442],
        [-0.9165956 , -0.23888063, -1.616482  , ..., -0.7969353 ,
          0.2658898 , -0.01221884],
        [ 0.05425656, -1.0066826 , -0.9775546 , ..., -0.23505326,
         -0.07000898,  0.03290154]]], dtype=float32)>

In [12]:
a =np.array(embedding[0][0])  #---> embedding of batch(here 1 sentence)
a 

array([[-0.52085036, -0.38367802, -2.135634  , ...,  0.13875636,
         0.25557533,  0.45156914],
       [ 0.15423784, -0.5098587 , -0.93862164, ..., -1.7609076 ,
        -0.07611521, -0.80006295],
       [-0.09277726, -0.00352648, -0.38873488, ..., -0.8360086 ,
         1.1227279 , -0.10558483],
       ...,
       [-0.87440735, -0.6651399 , -0.7555843 , ..., -0.44479614,
        -0.4471798 ,  0.38828442],
       [-0.9165956 , -0.23888063, -1.616482  , ..., -0.7969353 ,
         0.2658898 , -0.01221884],
       [ 0.05425656, -1.0066826 , -0.9775546 , ..., -0.23505326,
        -0.07000898,  0.03290154]], dtype=float32)

In [13]:
def bert_embedding(txt):
  idx = tokenizer.encode(txt,truncation=True) #creating tokens
  idx = np.array(idx)[None,:] #converting 2d array

  emb = bert(idx) #bert layer
  hidden = np.array(emb[0][0]) #batch output of last_hidden_state

  sent_emb = hidden.mean(0) # creating mean vector
  return sent_emb

In [14]:
num_sentences = 20000 #as I have limited computation power, this much sentences have been selected
sent_matrix = np.array([bert_embedding(text) for text in tqdm(df['Text'][0:num_sentences])])
sent_matrix.shape

  0%|          | 0/2832 [00:00<?, ?it/s]


KeyboardInterrupt



In [ ]:
positive_competence_emb = np.array([bert_embedding(t) for t in positive_competence])
positive_competence_emb

In [ ]:
positive_competence_emb.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
for d in [positive_competence,negative_competence,student_friendly,not_student_friendly,good_english,bad_english]:
  d_emb = np.array([bert_embedding(t) for t in d])
  print('Cosine similarity of given samples: {}'.format(cosine_similarity(d_emb[0][None,:],d_emb[1][None,:])))

In [ ]:
positive_competence_emb = np.array([bert_embedding(t) for t in positive_competence])
negative_competence_emb = np.array([bert_embedding(t) for t in negative_competence])
student_friendly_emb = np.array([bert_embedding(t) for t in student_friendly])
not_student_friendly_emb = np.array([bert_embedding(t) for t in not_student_friendly])
good_english_emb = np.array([bert_embedding(t) for t in good_english])
bad_english_emb = np.array([bert_embedding(t) for t in bad_english])

In [ ]:
cosine_score = pd.DataFrame(columns=['id','positive_competence','negative_competence','student_friendly','not_student_friendly','good_english','bad_english'])
cosine_score['id'] = range(len(sent_matrix))
cosine_score['positive_competence'] = cosine_similarity(sent_matrix,positive_competence_emb.mean(0)[None,:])
cosine_score['negative_competence'] = cosine_similarity(sent_matrix,negative_competence_emb.mean(0)[None,:])
cosine_score['student_friendly'] = cosine_similarity(sent_matrix,student_friendly_emb.mean(0)[None,:])
cosine_score['not_student_friendly'] = cosine_similarity(sent_matrix,not_student_friendly_emb.mean(0)[None,:])
cosine_score['good_english'] = cosine_similarity(sent_matrix,good_english_emb.mean(0)[None,:])
cosine_score['bad_english'] = cosine_similarity(sent_matrix,bad_english_emb.mean(0)[None,:])
cosine_score.head()

In [ ]:
df["positive_competence"]=cosine_similarity(sent_matrix,positive_competence_emb.mean(0)[None,:])
df["negative_competence"]=cosine_similarity(sent_matrix,negative_competence_emb.mean(0)[None,:])
df["student_friendly"]=cosine_similarity(sent_matrix,student_friendly_emb.mean(0)[None,:])
df["not_student_friendly"]=cosine_similarity(sent_matrix,not_student_friendly_emb.mean(0)[None,:])
df["good_english"]= cosine_similarity(sent_matrix,good_english_emb.mean(0)[None,:])
df["bad_english"]=cosine_similarity(sent_matrix,bad_english_emb.mean(0)[None,:])

In [ ]:
df.loc[df['Point']==1.0, 'positive_competence'] =df["positive_competence"]+0.05
df.loc[df['Point']==1.0, 'student_friendly'] =df["student_friendly"]+0.05
df.loc[df['Point']==1.0, 'good_english'] =df["good_english"]+0.05
df.loc[df['Point']==1.0, 'negative_competence'] =df["negative_competence"]-0.05
df.loc[df['Point']==1.0, 'not_student_friendly'] =df["not_student_friendly"]-0.05
df.loc[df['Point']==1.0, 'bad_english'] =df["bad_english"]-0.05
df.loc[df['Point']==0.0, 'positive_competence'] =df["positive_competence"]-0.05
df.loc[df['Point']==0.0, 'student_friendly'] =df["student_friendly"]-0.05
df.loc[df['Point']==0.0, 'good_english'] =df["good_english"]-0.05
df.loc[df['Point']==0.0, 'negative_competence'] =df["negative_competence"]+0.05
df.loc[df['Point']==0.0, 'not_student_friendly'] =df["not_student_friendly"]+0.05
df.loc[df['Point']==0.0, 'bad_english'] =df["bad_english"]+0.05
df["Competence"]=0.5+df["positive_competence"]-df["negative_competence"]
df["Friendly"]=0.5+df["student_friendly"]-df["not_student_friendly"]
df["English_Skill"]=0.5+df["good_english"]-df["bad_english"]
df=df.drop(['positive_competence', 'negative_competence',"good_english","bad_english","student_friendly","not_student_friendly"], axis=1)

In [ ]:
df.tail(2)

In [ ]:
df3=df2[df2.Text == "-"].copy()

In [ ]:
df3["Competence"]=""
df3["Friendly"]=""
df3["English_Skill"]=""
df3.loc[df3['Point']==1.0, 'Competence'] =0.600
df3.loc[df3['Point']==1.0, 'Friendly'] =0.600
df3.loc[df3['Point']==1.0, 'English_Skill'] =0.600
df3.loc[df3['Point']==0.5, 'Competence'] =0.500
df3.loc[df3['Point']==0.5, 'Friendly'] =0.500
df3.loc[df3['Point']==0.5, 'English_Skill'] =0.500
df3.loc[df3['Point']==0.0, 'Competence'] =0.400
df3.loc[df3['Point']==0.0, 'Friendly'] =0.400
df3.loc[df3['Point']==0.0, 'English_Skill'] =0.400
df3.head()

In [ ]:
df_final = pd.concat([df,df3])
df_final["Competence"]=pd.to_numeric(df_final["Competence"])
df_final["Friendly"]=pd.to_numeric(df_final["Friendly"])
df_final["English_Skill"]=pd.to_numeric(df_final["English_Skill"])

In [ ]:
df_final.head(2)

In [ ]:
df_group = df_final.groupby(["Lesson Name","Teacher Name"]).mean()
df_group.head(2)

In [ ]:
df_lessons=pd.read_csv("lessons_code.csv").copy()

In [ ]:
df_lessons.loc[df_lessons['Lesson Code'].str.contains(' '), 'Lesson Hours'] = df_lessons['Lesson Date']
df_lessons.loc[df_lessons['Lesson Code'].str.contains(' '), 'Lesson Date'] = df_lessons['Teacher Name']
df_lessons.loc[df_lessons['Lesson Code'].str.contains(' '), 'Teacher Name'] = '-'
df_lessons["Lesson Short"] = df_lessons["Lesson Code"].str.split('.').str[0]
df_lessons["Lesson Date"]=df_lessons["Lesson Date"].str[1:]
df_lessons["Lesson Hours"]=df_lessons["Lesson Hours"].str[1:]

In [ ]:
df_lessons.head(2)

In [ ]:
df_group = df_group.reset_index(level=-1)
df_group.head()

In [ ]:
df_group.rename(columns = {'Lesson Name':'Lesson Shorts'}, inplace = True)
df_final.rename(columns = {'Lesson Name':'Lesson Shorts'}, inplace = True)
df_group = df_group.reset_index(level=-1)
df_group["Teacher+Lesson"]= df_group["Teacher Name"]+" "+df_group["Lesson Name"]
df_lessons['Lesson Short'] = df_lessons['Lesson Short'].astype(str)
df_lessons["Teacher+Lesson"]= df_lessons["Teacher Name"]+" "+df_lessons["Lesson Short"]
df_group['Teacher+Lesson'] = df_group['Teacher+Lesson'].str.upper()
df_group['Teacher Name'] = df_group['Teacher Name'].str.upper()

In [ ]:
df_group.head(10)

In [ ]:
df_lessons.head(14)

In [ ]:
df_group['Teacher+Lesson'] = df_group['Teacher+Lesson'].str.replace('İ','I')
df_group['Teacher+Lesson'] = df_group['Teacher+Lesson'].str.replace('Ş','S')
df_group['Teacher+Lesson'] = df_group['Teacher+Lesson'].str.replace('Ö','O')
df_group['Teacher+Lesson'] = df_group['Teacher+Lesson'].str.replace('Ü','U')
df_group['Teacher+Lesson'] = df_group['Teacher+Lesson'].str.replace('Ç','C')
df_group['Teacher+Lesson'] = df_group['Teacher+Lesson'].str.replace('Ğ','G')
df_lessons['Teacher+Lesson'] = df_lessons['Teacher+Lesson'].str.replace('İ','I')
df_lessons['Teacher+Lesson'] = df_lessons['Teacher+Lesson'].str.replace('Ş','S')
df_lessons['Teacher+Lesson'] = df_lessons['Teacher+Lesson'].str.replace('Ö','O')
df_lessons['Teacher+Lesson'] = df_lessons['Teacher+Lesson'].str.replace('Ü','U')
df_lessons['Teacher+Lesson'] = df_lessons['Teacher+Lesson'].str.replace('Ç','C')
df_lessons['Teacher+Lesson'] = df_lessons['Teacher+Lesson'].str.replace('Ğ','G')

In [ ]:
df_merged = pd.merge(df_lessons, df_group, on='Teacher+Lesson')

In [ ]:
df_merged.head(10)

In [ ]:
df_merged = df_merged.drop(["Teacher Name_y","Lesson Name_y","Teacher+Lesson"],axis=1)

In [ ]:
df_merged.rename(columns = {'Lesson Name_x':'Lesson Name'}, inplace = True)
df_merged.rename(columns = {'Teacher Name_x':'Teacher Name'}, inplace = True)

In [ ]:
df_merged.head()

In [ ]:
with pd.ExcelWriter('final_excel_son2.xlsx') as writer:
    df_merged.to_excel(writer)